In [1]:
from models.xrag_models_utils import get_xrag_models

In [2]:
student_tokenizer, teacher_tokenizer, model = get_xrag_models()

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of XGemmaForCausalLM were not initialized from the model checkpoint at google/gemma-3-1b-it and are newly initialized: ['projector.projector.0.bias', 'projector.projector.0.weight', 'projector.projector.2.bias', 'projector.projector.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


initializing xRAG distillation...


In [ ]:
from configs.datasets import dataset as DatasetConfig
from configs.training import train_config as TrainConfig
from configs.distillation import distillation_config as DistillationConfig
from configs.fsdp import fsdp_config as FsdpConfig

In [4]:
dataset_config = DatasetConfig()
train_config = TrainConfig()
distill_config = DistillationConfig()
fsdp_config = DistillationConfig()
train_config.model_name = "google/gemma-3-1b-it"
distill_config.model_name = "Hannibal046/xrag-7b"
train_config.batch_size_training = 1
train_config.num_workers_dataloader = 1
dataset_config.file = "data/loaders/squad-v2-sampled.py"

In [8]:
from data.data_utils import get_dataloader
student_train_dataloader, student_eval_dataloader = get_dataloader(dataset_config, train_config, student_tokenizer, 0, distill_config)

--> Training Set Length = 1000
--> Validation Set Length = 250


In [9]:
from data.data_utils import get_dataloader
teacher_train_dataloader, teacher_eval_dataloader = get_dataloader(dataset_config, train_config, teacher_tokenizer, 0, distill_config)

--> Training Set Length = 1000
--> Validation Set Length = 250


In [10]:
student_item = next(iter(student_train_dataloader))
teacher_item = next(iter(teacher_train_dataloader))

In [38]:
type(student_item)

transformers.tokenization_utils_base.BatchEncoding

In [13]:
print(student_tokenizer.decode(student_item["input_ids"][0]))
print(teacher_tokenizer.decode(teacher_item["input_ids"][0]))

<start_of_turn>user
Background: <xRAG>, which also means:<end_of_turn>Internet service provider
[INST] Background:<xRAG>, which also means:[/INST] Internet service provider


In [ ]:
for key in teacher_item.keys():
    teacher_item[key] = teacher_item[key].to('cuda:0')

for key in student_item.keys():
    student_item[key] = student_item[key].to('cuda:0')

In [41]:
r = zip(student_item, teacher_item)

In [31]:
r2 = zip(student_train_dataloader, teacher_train_dataloader)

In [27]:
def forward(student_input_ids, student_attention_mask, student_retrieval_embeds, student_labels, teacher_input_ids, teacher_attention_mask, teacher_retrieval_embeds, teacher_labels):
    print("passed")

In [39]:
from models.distillation_model import preprocess_distillation_batch

In [44]:
for step, batch in enumerate(r2):
    batch = preprocess_distillation_batch(batch)
    for key in batch.keys():
        batch[key] = batch[key].to('cuda:0')
    # forward(**batch)
    student_output, teacher_output = model(**batch)
    if step == 1:
        break

In [45]:
student_output.loss

tensor(33.7881, device='cuda:0', grad_fn=<NllLossBackward0>)

In [46]:
teacher_output.loss

tensor(0.7753, device='cuda:0')